# FOSDEM Demo

This notebook presents a systematic movement data exploration protocol. 

## Setup

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [ ]:
FIGSIZE = (600,400)
SMSIZE = 300
COLOR = 'darkblue'
COLOR_HIGHLIGHT = 'red'
COLOR_BASE = 'grey'

In [ ]:
from math import sin, cos, atan2, radians, degrees, sqrt, pi
from datetime import datetime, date
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import datashader as ds
import holoviews as hv
from shapely.geometry import Point, LineString
from holoviews.operation.datashader import datashade, spread
from holoviews.element import tiles
from holoviews import opts, dim 
import hvplot
import movingpandas as mp
from shapely.geometry import Point

R_EARTH = 6371000  # radius of earth in meters
C_EARTH = 2 * R_EARTH * pi  # circumference
BG_TILES = tiles.CartoLight()

pd.set_option('use_inf_as_na', True)

In [ ]:
def plot_single_mover(df, mover_id, the_date):
    tmp = df[(df.id==mover_id) & (df.index.date==the_date)]
    gdf = gpd.GeoDataFrame(tmp.drop(['x', 'y'], axis=1), crs={'init': 'epsg:3857'}, geometry=[Point(xy) for xy in zip(tmp.x, tmp.y)])
    plot = mp.Trajectory(gdf, 1).hvplot(title=f'Mover {mover_id} ({the_date})', c='speed_m/s', cmap='RdYlBu',  colorbar=True, clim=(0,15), 
                                        line_width=5, width=FIGSIZE[0], height=FIGSIZE[1], tiles='CartoLight')
    return plot

In [ ]:
input_files = [
    'E:/Geodata/AISDK/raw_ais/aisdk_20170701.csv',
    'E:/Geodata/AISDK/raw_ais/aisdk_20170702.csv'
]

In [ ]:
df = pd.read_csv(input_files[0], nrows=100)
df.head()

In [ ]:
df['SOG'].hist(bins=100, figsize=(15,3))

In [ ]:
df = None
for input_file in input_files[:2]: 
    a = pd.read_csv(input_file, usecols=['# Timestamp', 'MMSI', 'Latitude', 'Longitude', 'SOG', 'Type of mobile', 'Ship type', 'Navigational status'])
    a = a[(a['Type of mobile'] == 'Class A') & (a.SOG>0)]
    a.drop(columns=['Type of mobile', 'SOG'], inplace=True)
    if df is None:
        df = a
    else:
        df = df.append(a)
    
df.rename(columns={'# Timestamp':'time', 'MMSI':'id', 'Latitude':'lat', 'Longitude':'lon', 'Ship type':'shiptype', 'Navigational status':'navstat'}, inplace=True)
df['time'] = pd.to_datetime(df['time'], format='%d/%m/%Y %H:%M:%S')

df.loc[:, 'x'], df.loc[:, 'y'] = ds.utils.lnglat_to_meters(df.lon, df.lat)

df.set_index('time', inplace=True)

df['navstat'] = df['navstat'].astype('category')
df['shiptype'] = df['shiptype'].astype('category')

df.head()

In [ ]:
print('Number of records: {} million'.format(round(len(df)/1000000)))

## A) Missing data


### A-1) Spatial gaps & outliers

#### Spatial spread / extent & outliers 


In [ ]:
print(f'Spatial extent: x_min={df.lon.min()}, x_max={df.lon.max()}, y_min={df.lat.min()}, y_max={df.lat.max()}')

In [ ]:
def plot_basic_scatter(df, color='darkblue', title='', width=FIGSIZE[0], height=FIGSIZE[1], size=2):
    opts.defaults(opts.Overlay(active_tools=['wheel_zoom']))
    pts = df.hvplot.scatter(x='x', y='y', datashade=True, cmap=[color, color], frame_width=width, frame_height=height, title=str(title))
    return BG_TILES * spread(pts, px=size)

In [ ]:
plot_basic_scatter(df, title='Spatial extent & outliers')

**Optional cropping of outliers**

In [ ]:
df = df[(df.lon>-90) & (df.lon<90) & (df.lat>0) & (df.lat<80)]

In [ ]:
cropped_df = df[(df.lon>0) & (df.lon<20) & (df.lat>52) & (df.lat<60)]
cropped_df['navstat'] = cropped_df['navstat'].astype('category')
cropped_df['shiptype'] = cropped_df['shiptype'].astype('category')
plot_basic_scatter(cropped_df)

#### Spatial gaps (selected areas / all movers / whole time span)



In [ ]:
def plot_point_density(df, title='', width=FIGSIZE[0], height=FIGSIZE[1]):
    opts.defaults(opts.Overlay(active_tools=['wheel_zoom']))
    pts = df.hvplot.scatter(x='x', y='y', title=str(title), datashade=True, frame_width=width, frame_height=height)
    return BG_TILES * pts

In [ ]:
plot_point_density(df, title='Spatial gaps')

### A-2) Temporal gaps & outliers

#### Temporal extent & outliers (whole territory / all movers / whole time span)



In [ ]:
print(f'Temporal extent: {df.index.min()} to {df.index.max()}')

In [ ]:
TIME_SAMPLE = '15min'

df['id'].resample(TIME_SAMPLE).count()\
    .hvplot(title=f'Number of records per {TIME_SAMPLE}', width=FIGSIZE[0])

#### Temporal gaps in linear sequence & temporal cycles (whole territory / all movers / time spans)


In [ ]:
counts_df = df['id'].groupby([df.index.hour, pd.Grouper(freq='d')]).count().to_frame(name='n')
counts_df.rename_axis(['hour', 'day'], inplace=True)
counts_df.hvplot.heatmap(title='Record count', x='hour', y='day', C='n', width=FIGSIZE[0])

### A-3) Spatiotemporal changes / gaps

#### Changing extent



In [ ]:
def plot_multiple_by_day(df, day, **kwargs):
    return plot_basic_scatter(df[df.index.date==day], title=day, width=SMSIZE, height=SMSIZE, **kwargs)
    
def plot_multiples_by_day(df, **kwargs):
    days = df.index.to_period('D').unique()
    a = None
    for a_day in days:
        a_day = a_day.to_timestamp().date()
        plot = plot_multiple_by_day(df, a_day, **kwargs)
        if a is None: a = plot
        else: a = a  + plot
    return a

In [ ]:
plot_multiples_by_day(cropped_df).cols(2)

### A-4) Attribute gaps


#### Spatial attribute gaps


In [ ]:
CATEGORY = 'shiptype' #'navstat'

cats = df[CATEGORY].unique()
#[cat for cat in cats]

In [ ]:
cmap = {} 
for cat in cats:
    cmap[cat] = COLOR_BASE
cmap['Unknown value'] = COLOR_HIGHLIGHT
cmap['Undefined'] = COLOR_HIGHLIGHT

In [ ]:
def plot_categorized_scatter(df, cat, title='', width=SMSIZE, height=SMSIZE, cmap=cmap):
    opts.defaults(opts.Overlay(active_tools=['wheel_zoom']))
    pts = df.hvplot.scatter(x='x', y='y', datashade=True, by=cat, colormap=cmap, legend=True, frame_width=width, frame_height=height, title=str(title))
    return BG_TILES * pts

In [ ]:
unknown = df[(df[CATEGORY]=='Unknown value') | (df[CATEGORY]=='Undefined')]
known = df[(df[CATEGORY]!='Unknown value') & (df[CATEGORY]!='Undefined')]

( plot_categorized_scatter(df, CATEGORY, title='Categorized', width=SMSIZE, height=SMSIZE, cmap=cmap) + 
  plot_basic_scatter(unknown, COLOR_HIGHLIGHT, title=f'Unknown {CATEGORY} only', width=SMSIZE, height=SMSIZE, size=1) +
  plot_basic_scatter(known, COLOR_BASE, title=f'Known {CATEGORY} only', width=SMSIZE, height=SMSIZE, size=1)
)

### DATA PREPARATION: Computing segment information

In [ ]:
def time_difference(row):
    t1 = row['prev_t']
    t2 = row['t']
    return (t2-t1).total_seconds()

def speed_difference(row):
    return row['speed_m/s'] - row['prev_speed']

def acceleration(row):
    if row['diff_t_s'] == 0:
        return None
    return row['diff_speed'] / row['diff_t_s']

def spherical_distance(lon1, lat1, lon2, lat2):
    delta_lat = radians(lat2 - lat1)
    delta_lon = radians(lon2 - lon1)
    a = sin(delta_lat/2) * sin(delta_lat/2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(delta_lon/2) * sin(delta_lon/2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    dist = R_EARTH * c
    return dist

def distance_to_prev(row):
    return spherical_distance(row['prev_lon'], row['prev_lat'], row['lon'], row['lat'])
    
def distance_to_next(row):
    return spherical_distance(row['next_lon'], row['next_lat'], row['lon'], row['lat'])

def direction(row):
    lon1, lat1, lon2, lat2 = row['prev_lon'], row['prev_lat'], row['lon'], row['lat']
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    delta_lon = radians(lon2 - lon1)
    x = sin(delta_lon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - (sin(lat1) * cos(lat2) * cos(delta_lon))
    initial_bearing = atan2(x, y)
    initial_bearing = degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360
    return compass_bearing

def angular_difference(row):
    diff = abs(row['prev_dir'] - row['dir'])
    if diff > 180:
        diff = abs(diff - 360)
    return diff 

def compute_segment_info(df):
    df = df.copy()
    df['t'] = df.index
    df['prev_t'] = df.groupby('id')['t'].shift()
    df['diff_t_s'] = df.apply(time_difference, axis=1)
    df['prev_lon'] = df.groupby('id')['lon'].shift()
    df['prev_lat'] = df.groupby('id')['lat'].shift()
    df['prev_x'] = df.groupby('id')['x'].shift()
    df['prev_y'] = df.groupby('id')['y'].shift()
    df['diff_x'] = df['x'] - df['prev_x']
    df['diff_y'] = df['y'] - df['prev_y']
    df['next_lon'] = df.groupby('id')['lon'].shift(-1)
    df['next_lat'] = df.groupby('id')['lat'].shift(-1)
    df['dist_prev_m'] = df.apply(distance_to_prev, axis=1)
    df['dist_next_m'] = df.apply(distance_to_next, axis=1)
    df['speed_m/s'] = df['dist_prev_m']/df['diff_t_s']
    df['prev_speed'] = df.groupby('id')['speed_m/s'].shift()
    df['diff_speed'] = df.apply(speed_difference, axis=1)
    df['acceleration'] = df.apply(acceleration, axis=1)
    df['dir'] = df.apply(direction, axis=1)
    df['prev_dir'] = df.groupby('id')['dir'].shift()
    df['diff_dir'] = df.apply(angular_difference, axis=1)
    df = df.drop(columns=['prev_x', 'prev_y', 'next_lon', 'next_lat', 'prev_speed', 'prev_dir'])
    return df

In [ ]:
%%time

try:
    segment_df = pd.read_pickle('./segments.pkl')
except:
    segment_df = compute_segment_info(cropped_df)
    segment_df.to_pickle("./segments.pkl")

### A-5) Gaps in trajectories

In [ ]:
GAP_MIN = 10000
GAP_MAX = 100000

segment_df['is_gap'] = ( (segment_df['dist_prev_m']>GAP_MIN) & (segment_df['dist_prev_m']<GAP_MAX) ) | ( (segment_df['dist_next_m']>GAP_MIN) & (segment_df['dist_next_m']<GAP_MAX) ) 
segment_df['id_by_gap'] = segment_df.groupby("id")['is_gap'].transform(lambda x: x.ne(x.shift()).cumsum())

In [ ]:
grouped = [df[['x','y']] for name, df in segment_df[segment_df.is_gap].groupby(['id', 'id_by_gap']) ]
path = hv.Path(grouped, kdims=['x','y'])
plot = datashade(path, cmap=COLOR_HIGHLIGHT).opts(frame_height=300, frame_width=300)
BG_TILES * plot

In [ ]:
segment_df.hvplot.scatter(title='Coordinate change plot', x='diff_x', y='diff_y', datashade=True, 
                          xlim=(-1000,1000), ylim=(-1000,1000), frame_width=FIGSIZE[1], frame_height=FIGSIZE[1])

In [ ]:
MINIMUM_NUMBER_OF_RECORDS = 100
MINIMUM_SPEED_MS = 1

def reset_values_at_daybreaks(tmp, columns):
    tmp['ix'] = tmp.index
    tmp['zero'] = 0
    ix_first = tmp.groupby(['id', pd.Grouper(freq='d')]).first()['ix']
    for col in columns:
        tmp[col] = tmp['zero'].where(tmp['ix'].isin(ix_first), tmp[col])
    tmp = tmp.drop(columns=['zero', 'ix'])
    return tmp

def compute_traj_info(segment_df):
    tmp = segment_df.copy()
    tmp['acceleration_abs'] = np.abs(tmp['acceleration'])
    tmp['diff_speed_abs'] = np.abs(tmp['diff_speed'])
    tmp = tmp.replace([np.inf, -np.inf], np.nan)

    tmp = reset_values_at_daybreaks(tmp, ['diff_t_s','dist_prev_m','diff_speed_abs','acceleration_abs'])

    traj_df = tmp.groupby(['id', pd.Grouper(freq='d')]) \
        .agg({'diff_t_s':['median', 'sum'], 
              'speed_m/s':['median','std'],
              'diff_dir':['median','std'], 
              'dist_prev_m':['median', 'sum'], 
              'diff_speed_abs':['max'], 
              'acceleration_abs':['median','max','mean','std'], 
              't':['min','count'],
             'shiptype':lambda x:x.value_counts().index[0]}) 

    traj_df.columns = ["_".join(x) for x in traj_df.columns.ravel()]
    traj_df = traj_df.rename(columns={'t_count':'n', 'shiptype_<lambda>':'shiptype', 
                                      'diff_t_s_sum':'duration_s', 'dist_prev_m_sum':'length_m'})
    traj_df['length_km'] = traj_df['length_m'] / 1000
    traj_df['duration_h'] = traj_df['duration_s'] / 3600
    traj_df['t_min_h'] = traj_df['t_min'].dt.hour + traj_df['t_min'].dt.minute / 60

    traj_df = traj_df[traj_df.n>=MINIMUM_NUMBER_OF_RECORDS]
    traj_df = traj_df[traj_df['speed_m/s_median']>=MINIMUM_SPEED_MS]
    return traj_df

In [ ]:
%%time

try:
    traj_df = pd.read_pickle('./traj.pkl')
except:
    traj_df = compute_traj_info(segment_df)
    traj_df.to_pickle("./traj.pkl")

In [ ]:
hvplot.scatter_matrix(
    traj_df[['length_km', 'speed_m/s_median', 'duration_h', 'diff_dir_median']]
)

### D-2) Spatial inaccuracy 


#### Outliers with unrealistic jumps




In [ ]:
segment_df['speed_m/s'].hvplot.hist(
    title='Histogram of speed between consecutive records', bins=100, frame_width=FIGSIZE[1], frame_height=250
) + segment_df.sort_values(by='speed_m/s', ascending=False)[:10][['id', 'speed_m/s']].hvplot.table(
    title='Top 10 records - speed', frame_width=FIGSIZE[1])

In [ ]:
plot_single_mover(segment_df, 218057000, date(2018,1,1))

#### Jitter / noise



In [ ]:
traj_df.hvplot.scatter(
    title='Direction difference median over standard deviation', alpha=0.3,
    x='diff_dir_median', y='diff_dir_std', hover_cols=['id','time'], #datashade=True,
    frame_width=FIGSIZE[1], frame_height=250, ylim=(-10,100)
) + traj_df.sort_values(by='diff_dir_median', ascending=False)[:10][['diff_dir_median','diff_dir_std']].hvplot.table(
    title='Top 10 trajectories - direction difference', frame_width=FIGSIZE[1])

In [ ]:
plot_single_mover(segment_df, 220614000, date(2018,1,1))